In [77]:
import os
import pandas as pd
from datetime import datetime
import json
import gc


# folder_path_demand = '/home/admin1/Downloads/punjab-data-prod-analysis/zirakpur/output_demand/'
folder_path_demanddetails = '/home/prerna/Punjab/punjab-data-prod-analysis/phagwara/output_demand_detail/'

# read active properties & needed columns
property_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/phagwara/eg_pt_property_phagwara_v2.csv',
    usecols=['id', 'propertyid', 'tenantid', 'createdtime', 'additionaldetails', 'ownershipcategory', 'status', 'usagecategory', 'landarea']
)
# property_df = property_df[property_df['status'] == 'ACTIVE'].copy()

# read units
# unit_df = pd.read_csv(
#     '/home/prerna/Punjab/punjab-data-prod-analysis/phagwara/eg_pt_unit_phagwara.csv',
#     usecols=['propertyid', 'occupancytype']
# )

address_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/phagwara/eg_pt_address_phagwara.csv',
    usecols=['propertyid', 'locality']
)

owner_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/phagwara/eg_pt_owner_phagwara.csv',
    usecols=['propertyid', 'userid']
)




# read demand
demand_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/phagwara/eg_demand_phagwara.csv',
    dtype={"consumercode": str},
    low_memory=False,
    usecols=['id', 'taxperiodfrom', 'taxperiodto', 'consumercode', 'status', 'businessservice']
)
demand_df = demand_df[demand_df['status'] == 'ACTIVE'].copy()
demand_df = demand_df[demand_df['businessservice'] == 'PT'].copy()

# read demand (memory‑efficient, in chunks)
# all_chunks = []
# needed_cols = ['id', 'taxperiodfrom', 'taxperiodto', 'consumercode', 'status', 'businessservice']
# for filename in os.listdir(folder_path_demand):
#     if filename.endswith('.csv'):
#         file_path = os.path.join(folder_path_demand, filename)
#         print(f'Loading: {file_path}')
#         chunk = pd.read_csv(file_path, usecols=needed_cols)
#         all_chunks.append(chunk)
# demand_df = pd.concat(all_chunks, ignore_index=True)
# demand_df = demand_df[demand_df['status'] == 'ACTIVE'].copy()
# demand_df = demand_df[demand_df['businessservice'] == 'PT'].copy()
# del all_chunks; gc.collect()

# read demand details (memory‑efficient, in chunks)
all_chunks = []
needed_cols = ['demandid', 'taxamount', 'collectionamount', 'taxheadcode']
for filename in os.listdir(folder_path_demanddetails):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path_demanddetails, filename)
        print(f'Loading: {file_path}')
        chunk = pd.read_csv(file_path, usecols=needed_cols)
        all_chunks.append(chunk)
demand_details_df = pd.concat(all_chunks, ignore_index=True)
del all_chunks; gc.collect()

print("✅ Loaded data")

Loading: /home/prerna/Punjab/punjab-data-prod-analysis/phagwara/output_demand_detail/output_4.csv
Loading: /home/prerna/Punjab/punjab-data-prod-analysis/phagwara/output_demand_detail/output_34.csv
Loading: /home/prerna/Punjab/punjab-data-prod-analysis/phagwara/output_demand_detail/output_38.csv
Loading: /home/prerna/Punjab/punjab-data-prod-analysis/phagwara/output_demand_detail/output_6.csv
Loading: /home/prerna/Punjab/punjab-data-prod-analysis/phagwara/output_demand_detail/output_60.csv
Loading: /home/prerna/Punjab/punjab-data-prod-analysis/phagwara/output_demand_detail/output_44.csv
Loading: /home/prerna/Punjab/punjab-data-prod-analysis/phagwara/output_demand_detail/output_17.csv
Loading: /home/prerna/Punjab/punjab-data-prod-analysis/phagwara/output_demand_detail/output_22.csv
Loading: /home/prerna/Punjab/punjab-data-prod-analysis/phagwara/output_demand_detail/output_59.csv
Loading: /home/prerna/Punjab/punjab-data-prod-analysis/phagwara/output_demand_detail/output_29.csv
Loading: /ho

In [78]:
count_paid = joined_demand['status'].value_counts()
print(count_paid)

status
ACTIVE    1811989
Name: count, dtype: int64


In [79]:
print(len(property_df))         # number of rows in properties
print(len(address_df))             # number of rows in units
print(len(owner_df))  
print(len(demand_df))   # number of rows in demand details
print(len(demand_details_df))   # number of rows in demand details

21958
40696
39589
142201
3248706


In [80]:
# join pt and unit
joined_pt_owner = property_df.merge(owner_df, left_on='id', right_on='propertyid', how='left', suffixes=('_property', '_owner'))
del property_df, owner_df; gc.collect()
joined_pt_owner.head()
print(joined_pt_owner['id'].nunique())

21958


In [66]:
joined_pt_owner.head()

,id,propertyid_property,tenantid,status,ownershipcategory,usagecategory,landarea,createdtime,additionaldetails,propertyid_owner,userid
0,dcd1e947-407d-4dce-bd12-56a426636fec,PT-1014-1035693,pb.phagwara,ACTIVE,INDIVIDUAL.SINGLEOWNER,MIXED,233.40,1724819902561,"{""inflammable"": null, ""businessName"": ""SHOP"", ...",dcd1e947-407d-4dce-bd12-56a426636fec,71bdbf29-5358-4c01-a90b-297a89be2490
1,4c1fa5a9-230e-481b-b8ad-329804877095,PT-1014-881544,pb.phagwara,ACTIVE,INDIVIDUAL.SINGLEOWNER,RESIDENTIAL,200.00,1591077941833,"{""remarks"": """", ""inflammable"": false, ""heightA...",4c1fa5a9-230e-481b-b8ad-329804877095,f0ee4588-1553-40c5-a53a-7cd5a2df4cdd
2,e3e66501-3669-474a-830f-50bcdeafd065,PT-1014-2110512,pb.phagwara,ACTIVE,INDIVIDUAL.MULTIPLEOWNERS,NONRESIDENTIAL.COMMERCIAL,66.67,1748924956892,"{""businessName"": ""COMM"", ""yearConstruction"": n...",e3e66501-3669-474a-830f-50bcdeafd065,8964d4de-3473-42dd-b2ad-30062e3dcfd3
3,e3e66501-3669-474a-830f-50bcdeafd065,PT-1014-2110512,pb.phagwara,ACTIVE,INDIVIDUAL.MULTIPLEOWNERS,NONRESIDENTIAL.COMMERCIAL,66.67,1748924956892,"{""businessName"": ""COMM"", ""yearConstruction"": n...",e3e66501-3669-474a-830f-50bcdeafd065,6ac1cb62-ab47-4f97-8ad7-3c2526c9bcbf
4,e3e66501-3669-474a-830f-50bcdeafd065,PT-1014-2110512,pb.phagwara,ACTIVE,INDIVIDUAL.MULTIPLEOWNERS,NONRESIDENTIAL.COMMERCIAL,66.67,1748924956892,"{""businessName"": ""COMM"", ""yearConstruction"": n...",e3e66501-3669-474a-830f-50bcdeafd065,93a13f5e-bcbf-4514-8728-915290dc7131


In [81]:
# join demand and demand details
joined_demand = demand_df.merge(demand_details_df, left_on='id', right_on='demandid', how='left', suffixes=('_demand', '_detail'))
print(joined_demand['id'].nunique())
del demand_details_df, demand_df; gc.collect()
joined_demand.head()

142201


,id,consumercode,businessservice,taxperiodfrom,taxperiodto,status,demandid,taxheadcode,taxamount,collectionamount
0,50c875db-c676-4214-abf4-907ebc2117d4,PT-1014-708136,PT,1396310400000,1427846399000,ACTIVE,50c875db-c676-4214-abf4-907ebc2117d4,PT_TIME_INTEREST,640.10,640.10
1,50c875db-c676-4214-abf4-907ebc2117d4,PT-1014-708136,PT,1396310400000,1427846399000,ACTIVE,50c875db-c676-4214-abf4-907ebc2117d4,PT_OWNER_EXEMPTION,0.00,0.00
2,50c875db-c676-4214-abf4-907ebc2117d4,PT-1014-708136,PT,1396310400000,1427846399000,ACTIVE,50c875db-c676-4214-abf4-907ebc2117d4,PT_CANCER_CESS,15.34,15.34
3,50c875db-c676-4214-abf4-907ebc2117d4,PT-1014-708136,PT,1396310400000,1427846399000,ACTIVE,50c875db-c676-4214-abf4-907ebc2117d4,PT_TAX,766.67,766.67
4,50c875db-c676-4214-abf4-907ebc2117d4,PT-1014-708136,PT,1396310400000,1427846399000,ACTIVE,50c875db-c676-4214-abf4-907ebc2117d4,PT_UNIT_USAGE_EXEMPTION,0.00,0.00


In [82]:
# Calculate sums per demand id
paid_df = (
    joined_demand.groupby('id', as_index=False)
    .agg({
        'consumercode': 'first',  # keep consumercode value
        'taxamount': 'sum',
        'collectionamount': 'sum'
    })
)

# Add Paid flag based on difference
paid_df['Paid'] = paid_df.apply(
    lambda row: 'Yes' if (row['taxamount'] - row['collectionamount']) <= 0 else 'No',
    axis=1
)

# Merge back to original DF so every row has the Paid column
joined_demand = joined_demand.merge(
    paid_df[['id', 'Paid']],
    on='id',
    how='left'
)

print(joined_demand.head())

                                     id    consumercode businessservice  \
0  50c875db-c676-4214-abf4-907ebc2117d4  PT-1014-708136              PT   
1  50c875db-c676-4214-abf4-907ebc2117d4  PT-1014-708136              PT   
2  50c875db-c676-4214-abf4-907ebc2117d4  PT-1014-708136              PT   
3  50c875db-c676-4214-abf4-907ebc2117d4  PT-1014-708136              PT   
4  50c875db-c676-4214-abf4-907ebc2117d4  PT-1014-708136              PT   

   taxperiodfrom    taxperiodto  status                              demandid  \
0  1396310400000  1427846399000  ACTIVE  50c875db-c676-4214-abf4-907ebc2117d4   
1  1396310400000  1427846399000  ACTIVE  50c875db-c676-4214-abf4-907ebc2117d4   
2  1396310400000  1427846399000  ACTIVE  50c875db-c676-4214-abf4-907ebc2117d4   
3  1396310400000  1427846399000  ACTIVE  50c875db-c676-4214-abf4-907ebc2117d4   
4  1396310400000  1427846399000  ACTIVE  50c875db-c676-4214-abf4-907ebc2117d4   

               taxheadcode  taxamount  collectionamount Paid  

In [69]:
no_paid_ids = paid_df.loc[paid_df['Paid'] == 'No', 'id']
print(no_paid_ids)

4         0000f4d5-6670-4168-8f71-aa4d9029a6bf
14        0008d686-ceef-4a52-a6b5-6af4899206ea
15        000936bd-a63c-4dd5-851c-06f41d0bb063
17        0009c2e8-4b1c-475f-84da-8bcc862ec8c4
24        000f256c-5783-4760-b050-81be8357c610
                          ...                 
142176    fff3a33c-74a6-4345-9279-f5d5a3f5857a
142179    fff47be6-af11-4b7d-a3f9-e3f2aee50b92
142180    fff4acf5-d277-4259-aba7-ecafb109a1c2
142189    fff9eecd-137c-4e68-b8bd-05a1a8c3b929
142191    fffa52fc-a251-4340-b193-33ef1a39ca32
Name: id, Length: 41557, dtype: object


In [83]:

# Step 1: Convert to datetime and add FY column
joined_demand['taxperiodfrom_dt'] = pd.to_datetime(joined_demand['taxperiodfrom'], unit='ms', utc=True)

# Convert to IST (Asia/Kolkata)
ist = pytz.timezone('Asia/Kolkata')
joined_demand['taxperiodfrom_dt'] = joined_demand['taxperiodfrom_dt'].dt.tz_convert(ist)

def get_fy(date):
    if date.month >= 4:
        fy_start = date.year
        fy_end = date.year + 1
    else:
        fy_start = date.year - 1
        fy_end = date.year
    return f"{fy_start}-{str(fy_end)[-2:]}"

joined_demand['fy'] = joined_demand['taxperiodfrom_dt'].apply(get_fy)

# Step 2: Determine Yes/No/Partial for each consumercode & FY
def fy_paid_status(values):
    unique_vals = set(values)
    if len(unique_vals) > 1:
        return "Partial"
    else:
        return list(unique_vals)[0]

status_df = (
    joined_demand.groupby(['consumercode', 'fy'])['Paid']
    .apply(fy_paid_status)
    .reset_index()
)

# Step 3: Pivot FYs into separate columns
pivot_df = status_df.pivot(index='consumercode', columns='fy', values='Paid').reset_index()

print(pivot_df)


fy        consumercode 2013-14 2014-15 2015-16 2016-17 2017-18 2018-19  \
0       PT-1014-052724     NaN     NaN     NaN     NaN     NaN     Yes   
1       PT-1014-067025     NaN     NaN     NaN     NaN     NaN     NaN   
2      PT-1014-1000000     NaN     NaN     NaN     NaN     NaN     NaN   
3      PT-1014-1000004     NaN     NaN     NaN     NaN     NaN     NaN   
4      PT-1014-1000006     NaN     NaN     NaN     NaN     NaN     NaN   
...                ...     ...     ...     ...     ...     ...     ...   
21778   PT-1014-999754     NaN     NaN     NaN     NaN     NaN     NaN   
21779   PT-1014-999756     NaN     NaN     NaN     NaN     NaN     NaN   
21780   PT-1014-999820     NaN     NaN     NaN     NaN     NaN     NaN   
21781   PT-1014-999823     NaN     NaN     NaN     NaN     NaN     NaN   
21782   PT-1014-999928     NaN     NaN     NaN     NaN     NaN     NaN   

fy    2019-20 2020-21 2021-22 2022-23 2023-24 2024-25 2025-26  
0         NaN     Yes     Yes     Yes     Yes  

In [84]:
# Merge pivot_df with joined_pt_owner
merged_df = joined_pt_owner.merge(
    pivot_df,
    left_on='propertyid_property',  # from joined_pt_owner
    right_on='consumercode',        # from pivot_df
    how='left'
)

# Drop 'consumercode' column from the merge if you don’t need it
merged_df.drop(columns=['consumercode'], inplace=True)

print(merged_df.head())

                                     id propertyid_property     tenantid  \
0  dcd1e947-407d-4dce-bd12-56a426636fec     PT-1014-1035693  pb.phagwara   
1  4c1fa5a9-230e-481b-b8ad-329804877095      PT-1014-881544  pb.phagwara   
2  e3e66501-3669-474a-830f-50bcdeafd065     PT-1014-2110512  pb.phagwara   
3  e3e66501-3669-474a-830f-50bcdeafd065     PT-1014-2110512  pb.phagwara   
4  e3e66501-3669-474a-830f-50bcdeafd065     PT-1014-2110512  pb.phagwara   

   status          ownershipcategory              usagecategory  landarea  \
0  ACTIVE     INDIVIDUAL.SINGLEOWNER                      MIXED    233.40   
1  ACTIVE     INDIVIDUAL.SINGLEOWNER                RESIDENTIAL    200.00   
2  ACTIVE  INDIVIDUAL.MULTIPLEOWNERS  NONRESIDENTIAL.COMMERCIAL     66.67   
3  ACTIVE  INDIVIDUAL.MULTIPLEOWNERS  NONRESIDENTIAL.COMMERCIAL     66.67   
4  ACTIVE  INDIVIDUAL.MULTIPLEOWNERS  NONRESIDENTIAL.COMMERCIAL     66.67   

     createdtime                                  additionaldetails  \
0  172481

In [72]:
print(merged_df.columns)

Index(['id', 'propertyid_property', 'tenantid', 'status', 'ownershipcategory',
       'usagecategory', 'landarea', 'createdtime', 'additionaldetails',
       'propertyid_owner', 'userid', '2013-14', '2014-15', '2015-16',
       '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22',
       '2022-23', '2023-24', '2024-25', '2025-26'],
      dtype='object')


In [85]:
# Identify all FY year columns
fy_cols = [col for col in merged_df.columns if col.startswith("20") and "-" in col]

# Group and aggregate
grouped_df = (
    merged_df
    .groupby(["propertyid_property"] + fy_cols, dropna=False)
    .agg({
        "id" : "first",
        "landarea": "first",               # keep first value from property df
        "usagecategory": "first",           # keep first value from property df
        "userid": lambda x: list(set(x))
    })  # unique list of owners
    .reset_index()
)

# Optional: rename userid column
grouped_df.rename(columns={"userid": "userids_list"}, inplace=True)

print(grouped_df)


      propertyid_property 2013-14 2014-15 2015-16 2016-17 2017-18 2018-19  \
0          PT-1014-052724     NaN     NaN     NaN     NaN     NaN     Yes   
1         PT-1014-1000000     NaN     NaN     NaN     NaN     NaN     NaN   
2         PT-1014-1000004     NaN     NaN     NaN     NaN     NaN     NaN   
3         PT-1014-1000006     NaN     NaN     NaN     NaN     NaN     NaN   
4         PT-1014-1000007     NaN     NaN     NaN     NaN     NaN     NaN   
...                   ...     ...     ...     ...     ...     ...     ...   
21953      PT-1014-999754     NaN     NaN     NaN     NaN     NaN     NaN   
21954      PT-1014-999756     NaN     NaN     NaN     NaN     NaN     NaN   
21955      PT-1014-999820     NaN     NaN     NaN     NaN     NaN     NaN   
21956      PT-1014-999823     NaN     NaN     NaN     NaN     NaN     NaN   
21957      PT-1014-999928     NaN     NaN     NaN     NaN     NaN     NaN   

      2019-20 2020-21 2021-22 2022-23 2023-24 2024-25 2025-26  \
0         

In [75]:
print(len(grouped_df))

21958


In [87]:
pt_address_df = pd.read_csv(
    '/home/prerna/Punjab/punjab-data-prod-analysis/phagwara/eg_pt_address_phagwara.csv',
    usecols=['propertyid', 'locality']
)
print(len(pt_address_df))
print(pt_address_df)

40696
                                 propertyid locality
0      9dc74eb1-9dfa-49df-b0f5-a3a3acc4e5da    PLC52
1      a9e2ca6a-4ab3-4d14-9ccf-390056bd7598   PLC114
2      ee87180f-0a8f-491f-9297-f79f9ea7f6e1    PLC98
3      5f5ab5ef-3d51-46cb-9f9c-be58f9ca71a7   PLC154
4      7a98c802-3396-421c-a1ef-27395416e78f   PLC163
...                                     ...      ...
40691  1bdf5005-ffa0-4590-9166-b1197efa2374    PLC74
40692  b4feaa92-ad08-48ad-ba3d-3d908c737b1f   PLC142
40693  44abb2c6-7198-41b7-baba-278eba6eea00   PLC142
40694  1543ba52-87ce-43ae-8e10-aee13b7eec92   PLC131
40695  0fdf2a12-c1e7-4bf2-aa55-cdda42c16eb1   PLC142

[40696 rows x 2 columns]


In [89]:
dup_counts = (
    pt_address_df
    .groupby(['propertyid', 'locality'])
    .size()
    .reset_index(name='count')
    .query('count > 1')  # Keep only duplicates
)

print(dup_counts)
print(f"Total duplicate combinations: {len(dup_counts)}")

                                 propertyid locality  count
5      000fc31c-97c7-417b-bf6b-43aab3c1c026   PLC153      3
8      001237bc-02ec-4429-925b-e60cbd507542    PLC68      3
17     00256c2c-0ca2-4531-a3b1-f69987171d27   PLC180      4
26     003846c2-3141-40be-8158-f3067e138257   PLC123      2
31     00418487-f6fa-492c-bd87-e7e21e028f2e    PLC13      2
...                                     ...      ...    ...
34157  ff5f2699-4ae7-41ae-96b5-ab57244e23ad    PLC13      2
34189  ffa7dbf8-ebc5-4773-9aba-b701f945af26    PLC52      2
34197  ffb76054-4582-454d-b1b8-b71b591c41af   PLC142      2
34216  ffde9b83-0ffd-4c75-ba12-2abe94465041   PLC197      3
34226  fffa6836-6c8b-4cf0-b36f-1ba380222ce0    PLC20      3

[2761 rows x 3 columns]
Total duplicate combinations: 2761


In [90]:
# Collapse duplicates: each propertyid gets a set of localities
address_grouped = (
    pt_address_df
    .groupby('propertyid', dropna=False)['locality']
    .agg(lambda x: list(sorted(set(x.dropna()))))  # unique localities as list
    .reset_index()
)

# Merge back to grouped_df
final_df = grouped_df.merge(
    address_grouped,
    left_on='id',
    right_on='propertyid',
    how='left'
)

# Drop the extra join column
final_df.drop(columns=['propertyid'], inplace=True)

print(final_df.head())
print(len(final_df))

  propertyid_property 2013-14 2014-15 2015-16 2016-17 2017-18 2018-19 2019-20  \
0      PT-1014-052724     NaN     NaN     NaN     NaN     NaN     Yes     NaN   
1     PT-1014-1000000     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2     PT-1014-1000004     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3     PT-1014-1000006     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4     PT-1014-1000007     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

  2020-21 2021-22 2022-23 2023-24 2024-25 2025-26  \
0     Yes     Yes     Yes     Yes     Yes     Yes   
1     Yes     Yes     Yes     Yes     Yes     Yes   
2     Yes     Yes     Yes     Yes     Yes     Yes   
3     Yes     Yes     Yes     Yes     Yes     Yes   
4     Yes     Yes     Yes     Yes     Yes      No   

                                     id  landarea usagecategory  \
0  34c7e455-d9ce-452f-9c2e-05bafb5f195c     378.0   RESIDENTIAL   
1  c2c9ed10-82b3-45b8-a756-9c7d8b8e1360      25.0   RESIDENT

In [92]:
fy_cols = [col for col in final_df.columns if col.startswith("20") and "-" in col]
fy_cols_sorted = sorted(fy_cols)  # ensure chronological order

def mark_gaps(row):
    seen_demand = False
    statuses = {}
    for col in fy_cols_sorted:
        val = row[col]
        if pd.notna(val):
            seen_demand = True
            statuses[col + "_status"] = "Demand generated"
        else:
            if seen_demand:
                future_has_demand = any(pd.notna(row[c]) for c in fy_cols_sorted[fy_cols_sorted.index(col)+1:])
                if future_has_demand:
                    statuses[col + "_status"] = "Demand not generated, hence not paid"
                else:
                    statuses[col + "_status"] = "No demand"
            else:
                statuses[col + "_status"] = "No demand"
    return pd.Series(statuses)

status_df = final_df.apply(mark_gaps, axis=1)
merged_with_status = pd.concat([final_df, status_df], axis=1)

print(merged_with_status.head(10))
print(len(merged_with_status))


  propertyid_property 2013-14 2014-15 2015-16 2016-17 2017-18 2018-19 2019-20  \
0      PT-1014-052724     NaN     NaN     NaN     NaN     NaN     Yes     NaN   
1     PT-1014-1000000     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2     PT-1014-1000004     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3     PT-1014-1000006     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4     PT-1014-1000007     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5     PT-1014-1000075     NaN     NaN     NaN     NaN     Yes     Yes     Yes   
6     PT-1014-1000137     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
7     PT-1014-1000293     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
8     PT-1014-1000302     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
9     PT-1014-1000313     NaN     NaN     Yes     Yes     Yes     Yes     Yes   

  2020-21 2021-22  ...    2016-17_status    2017-18_status    2018-19_status  \
0     Yes     Yes  ...      

In [93]:
import numpy as np

fy_cols = [col for col in final_df.columns if col.startswith("20") and "-" in col]
fy_cols_sorted = sorted(fy_cols)  # ensure chronological order

def fill_gaps(row):
    seen_demand = False
    for col in fy_cols_sorted:
        if pd.notna(row[col]):  
            seen_demand = True
        elif seen_demand:  
            # Only mark NaNs if demand was already seen before this FY
            # and later we might see demand again
            # We'll check forward to confirm
            future_has_demand = any(pd.notna(row[c]) for c in fy_cols_sorted[fy_cols_sorted.index(col)+1:])
            if future_has_demand:
                row[col] = "Demand not generated, hence not paid"
    return row

final_df[fy_cols_sorted] = final_df[fy_cols_sorted].apply(fill_gaps, axis=1)

print(final_df[fy_cols_sorted].head(10))


  2013-14 2014-15 2015-16 2016-17 2017-18 2018-19  \
0     NaN     NaN     NaN     NaN     NaN     Yes   
1     NaN     NaN     NaN     NaN     NaN     NaN   
2     NaN     NaN     NaN     NaN     NaN     NaN   
3     NaN     NaN     NaN     NaN     NaN     NaN   
4     NaN     NaN     NaN     NaN     NaN     NaN   
5     NaN     NaN     NaN     NaN     Yes     Yes   
6     NaN     NaN     NaN     NaN     NaN     NaN   
7     NaN     NaN     NaN     NaN     NaN     NaN   
8     NaN     NaN     NaN     NaN     NaN     NaN   
9     NaN     NaN     Yes     Yes     Yes     Yes   

                                2019-20 2020-21 2021-22 2022-23 2023-24  \
0  Demand not generated, hence not paid     Yes     Yes     Yes     Yes   
1                                   NaN     Yes     Yes     Yes     Yes   
2                                   NaN     Yes     Yes     Yes     Yes   
3                                   NaN     Yes     Yes     Yes     Yes   
4                                   NaN  

In [94]:
print(final_df)

      propertyid_property 2013-14 2014-15 2015-16 2016-17 2017-18 2018-19  \
0          PT-1014-052724     NaN     NaN     NaN     NaN     NaN     Yes   
1         PT-1014-1000000     NaN     NaN     NaN     NaN     NaN     NaN   
2         PT-1014-1000004     NaN     NaN     NaN     NaN     NaN     NaN   
3         PT-1014-1000006     NaN     NaN     NaN     NaN     NaN     NaN   
4         PT-1014-1000007     NaN     NaN     NaN     NaN     NaN     NaN   
...                   ...     ...     ...     ...     ...     ...     ...   
21953      PT-1014-999754     NaN     NaN     NaN     NaN     NaN     NaN   
21954      PT-1014-999756     NaN     NaN     NaN     NaN     NaN     NaN   
21955      PT-1014-999820     NaN     NaN     NaN     NaN     NaN     NaN   
21956      PT-1014-999823     NaN     NaN     NaN     NaN     NaN     NaN   
21957      PT-1014-999928     NaN     NaN     NaN     NaN     NaN     NaN   

                                    2019-20 2020-21  \
0      Demand not ge

In [96]:
report = final_df.rename(columns={
    'propertyid_property': 'Property ID',
    '2013-14' : '2013-14',
    '2014-15' : '2014-15',
    '2015-16' : '2015-16',
    '2016-17' : '2016-17',
    '2017-18' : '2017-18',
    '2018-19' : '2018-19',
    '2019-20' : '2019-20',
    '2020-21' : '2020-21',
    '2021-22' : '2021-22',
    '2022-23' : '2022-23',
    '2023-24' : '2023-24',
    '2024-25' : '2024-25',
    '2025-26' : '2025-26',
    'landarea' : 'Plot size',
    'usagecategory' : 'Propertyusagetype',
    'locality' : 'Locality',
    'userids_list' : 'Owner Id'
}).copy()

# then keep only those columns, in the exact order you want:
final_report = report[
    [
        'Property ID',
        'Owner Id',
        'Locality',
        'Plot size',
        'Propertyusagetype',
        '2013-14',
        '2014-15',
        '2015-16',
        '2016-17',
        '2017-18',
        '2018-19',
        '2019-20',
        '2020-21',
        '2021-22',
        '2022-23',
        '2023-24',
        '2024-25'
    ]
].copy()

In [97]:
final_report.to_csv('Punjab_Data_Analysis_Phagwara_PDC.csv', index=False)